In [1]:
# imports
import pandas as pd

In [17]:
# create column names
features = []
for i in range (1,434):
    feature_name = f"feature_{i}"
    # print(feature_name)
    features.append(feature_name)
# features.append("label")
len(features)

433

In [28]:
# merge test data and labels
def merge(data, labels):
    dataDf = pd.read_csv(data, header=None, names=features, float_precision='round_trip')
    labelsDf = pd.read_csv(labels, header=None, names=['label'])
    merged = pd.merge(dataDf, labelsDf, left_index=True, right_index=True)
    print(f"data: {dataDf.shape}, labels: {labelsDf.shape}, merged: {merged.shape}")
    print(f"{data}: \n{dataDf.iloc[4]}")
    return merged

test_merged = merge("original_data/noExclusion_test_data.csv","original_data/noExclusion_test_label.csv")
train_merged = merge("original_data/noExclusion_train_data.csv","original_data/noExclusion_train_label.csv")    
# print(f"test: {test_merged.shape}, train: {train_merged.shape}")
combined = pd.concat([train_merged,test_merged])
combined.to_csv('combined.csv')
test_merged.to_csv('test_merged.csv')
train_merged.to_csv('train_merged.csv')


data: (143, 433), labels: (143, 1), merged: (143, 434)
original_data/noExclusion_test_data.csv: 
feature_1      0.148980
feature_2      0.147850
feature_3      0.150650
feature_4      0.151850
feature_5      0.153540
                 ...   
feature_429    0.063371
feature_430    0.063117
feature_431    0.061872
feature_432    0.059312
feature_433    0.061870
Name: 4, Length: 433, dtype: float64
data: (572, 433), labels: (572, 1), merged: (572, 434)
original_data/noExclusion_train_data.csv: 
feature_1      0.739760
feature_2      0.727150
feature_3      0.713110
feature_4      0.697040
feature_5      0.671250
                 ...   
feature_429    0.104410
feature_430    0.098250
feature_431    0.099586
feature_432    0.092213
feature_433    0.087415
Name: 4, Length: 433, dtype: float64
